# CSE 158, Fall 2019: Assignment 1 - Category

**Name:** CUI, HONGJIAN &ensp;&ensp;&ensp;&ensp; **PID:** U08398995 

In [1]:
import matplotlib.pyplot as plt
import json
import numpy as np
import random
import string
from collections import defaultdict
from nltk.stem.porter import PorterStemmer
from nltk.corpus import stopwords
from sklearn import linear_model

In [2]:
def parseData(fname):
    for l in open(fname):
        yield eval(l)

In [3]:
path = "/home/cui/Projects/PycharmProjects/CSE-158/data/train_Category.json"

In [4]:
dataset = list(parseData(path))

In [5]:
dataset[0]

{'n_votes': 0,
 'review_id': 'r99763621',
 'user_id': 'u17334941',
 'review_text': "Genuinely enthralling. If Collins or Bernard did invent this out of whole cloth, they deserve a medal for imagination. Lets leave the veracity aside for a moment - always a touchy subject when it comes to real life stories of the occult - and talk about the contents. \n The Black Alchemist covers a period of two years in which Collins, a magician, and Bernard, a psychic, undertook a series of psychic quests that put them in opposition with the titular Black Alchemist. As entertainment goes, the combination of harrowing discoveries, ancient lore, and going down the pub for a cigarette and a Guinness, trying to make sense of it all while a hen party screams at each other, is a winner. It is simultaneously down to earth and out of this world. \n It reads fast, both because of the curiousity and because Collins has a very clear writing style. Sometimes its a little clunky or over repetitive and there's a fe

In [6]:
split = 190000

In [7]:
training_set = dataset[:split]
validation_set = dataset[split:]

In [8]:
# import nltk
# nltk.download('stopwords')

In [9]:
wordCounts = defaultdict(int)
punctuation = set(string.punctuation)
totalWords = 0
stemmer = PorterStemmer()
stopwords = stopwords.words("english")
corpus = defaultdict(set)

In [10]:
index = 0
for d in training_set:
    text = d['review_text']
    text = text.lower()
    text = [c for c in text if not (c in punctuation)]
    text = ''.join(text)
    words = text.strip().split()
    for word in words:
        w = stemmer.stem(word)
        if w in stopwords: continue
        totalWords += 1
        wordCounts[w] += 1
        corpus[index].add(w)
    index += 1

In [11]:
def IDF(wordSet, wordId):
    dataCount = len(corpus)
    IDF = np.zeros(len(wordSet), dtype="float16")

    for d in corpus:
        review = corpus[d]
        wordsAppeared = set()
        for word in review:
            if word in wordSet:
                wordsAppeared.add(word)
        for word in wordsAppeared:
            IDF[wordId[word]] += 1

    IDF = np.log(dataCount / IDF)
    return IDF

In [12]:
def getIDF(n):
    popularWords = sorted(wordCounts.items(), key=lambda x:x[1])[-n:]
    popularWords.reverse()
    wordSet = [w[0] for w in popularWords]
    wordId = dict(zip(wordSet, range(len(wordSet))))
    
    return IDF(wordSet, wordId)

In [13]:
IDF = getIDF(4300)

In [14]:
del corpus

In [15]:
def feature(datum): 
    text = datum['review_text']
    text = text.lower()
    text = [c for c in text if not (c in punctuation)]
    text = ''.join(text)
    
    tf = np.zeros(len(wordSet), dtype="float16")
    word = text.strip().split()
    count = len(word)
    for w in word:
        w = stemmer.stem(w)
        if not (w in wordSet): continue
        if w in stopwords: continue
        tf[wordId[w]] += 1
    
    tf = tf / count
    idf = IDF[:len(wordSet)]
    
    feat = tf * idf
    
    return feat

In [15]:
dic_size = [2100, 2400, 2700, 3000]
C = [1, 10, 100]
accuracyList = []

y_train = np.array([d['genreID'] for d in training_set])
y_validation = np.array([d['genreID'] for d in validation_set])

for s in dic_size:
    print("Dictionary size: " + str(s))
    
    popularWords = sorted(wordCounts.items(), key=lambda x:x[1])[-s:]
    popularWords.reverse()
    wordSet = [w[0] for w in popularWords]
    wordId = dict(zip(wordSet, range(len(wordSet))))
    
    X_train = np.array([feature(d) for d in training_set], dtype="float32")
    X_validation = np.array([feature(d) for d in validation_set], dtype="float32")
    
    for c in C:
        print("C = " + str(c))
    
        model = linear_model.LogisticRegression(solver='lbfgs', C=c, multi_class='auto', max_iter=2000)
        model.fit(X_train, y_train)
        validation_predictions = model.predict(X_validation)
        correctPredictionsValidation = validation_predictions == y_validation
        accuracy = sum(correctPredictionsValidation) / len(correctPredictionsValidation)
    
        accuracyList.append([s, c, accuracy])

Dictionary size: 2100
C = 1
C = 10
C = 100


/home/cui/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:947: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)


Dictionary size: 2400
C = 1
C = 10
C = 100


/home/cui/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:947: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)


Dictionary size: 2700
C = 1
C = 10
C = 100


KeyboardInterrupt: 

In [16]:
accuracyList

[[2100, 1, 0.6911],
 [2100, 10, 0.7103],
 [2100, 100, 0.7088],
 [2400, 1, 0.6934],
 [2400, 10, 0.7159],
 [2400, 100, 0.7134],
 [2700, 1, 0.6985],
 [2700, 10, 0.7198]]

In [94]:
accuracyList

[[300, 0.001, 0.3603],
 [300, 0.01, 0.3669],
 [300, 0.1, 0.4644],
 [300, 1, 0.5399],
 [300, 10, 0.5556],
 [300, 100, 0.5582],
 [600, 0.001, 0.3603],
 [600, 0.01, 0.374],
 [600, 0.1, 0.5289],
 [600, 1, 0.6272],
 [600, 10, 0.6457],
 [600, 100, 0.6503],
 [900, 0.001, 0.3603],
 [900, 0.01, 0.3757],
 [900, 0.1, 0.5494],
 [900, 1, 0.6497],
 [900, 10, 0.6669],
 [900, 100, 0.6688],
 [1200, 0.001, 0.3603],
 [1200, 0.01, 0.3771],
 [1200, 0.1, 0.5633],
 [1200, 1, 0.6649],
 [1200, 10, 0.681],
 [1200, 100, 0.6837],
 [1500, 0.001, 0.3603],
 [1500, 0.01, 0.3783],
 [1500, 0.1, 0.5725],
 [1500, 1, 0.6801],
 [1500, 10, 0.6958],
 [1500, 100, 0.6977],
 [1800, 0.001, 0.3603],
 [1800, 0.01, 0.3799],
 [1800, 0.1, 0.5782],
 [1800, 1, 0.6856],
 [1800, 10, 0.7036],
 [1800, 100, 0.7066]]

In [1]:
def draw(acc_list, dic_size):
    plt.figure(figsize=(15,8))
    
    y_plt = [d[2] for d in acc_list if d[0] == dic_size]
    
    x_axis = [100 / len(C) * C.index(x) for x in C]

    plt.plot(x_axis, y_plt)
    
    plt.xticks(x_axis, [str(c) for c in C])
    y_max = max(y_plt)
    x_max = x_axis[y_plt.index(y_max)]
    plt.text(x_max, y_max, [C[y_plt.index(y_max)], y_max],ha='center', va='bottom', fontsize=15)

    plt.title("The accuracy of different C with dictionary size = " + str(dic_size))
    plt.xlabel("C")
    plt.ylabel("Accuracy")
    plt.show()

In [16]:
popularWords = sorted(wordCounts.items(), key=lambda x:x[1])[-4300:]
popularWords.reverse()
wordSet = [w[0] for w in popularWords]
wordId = dict(zip(wordSet, range(len(wordSet))))

X_train = np.array([feature(d) for d in training_set[:150000]], dtype="float16")
X_validation = np.array([feature(d) for d in validation_set], dtype="float16")
y_train = np.array([d['genreID'] for d in training_set[:150000]], dtype="float16")
y_validation = np.array([d['genreID'] for d in validation_set], dtype="float16")

In [17]:
model = linear_model.LogisticRegression(solver='lbfgs', C=100, multi_class='auto', max_iter=3000, verbose=1)
model.fit(X_train, y_train)
pred = model.predict(X_validation)
prediction = pred == y_validation
print(sum(prediction) / len(prediction))

dataset_test = list(parseData("/home/cui/Projects/PycharmProjects/CSE-158/data/test_Category.json"))

predictions = open("/home/cui/Projects/PycharmProjects/CSE-158/data/predictions_Category_4000.txt", 'w')
predictions.write("userID-reviewID,prediction\n")

for l in dataset_test:
    cat = model.predict([feature(l)])
    predictions.write(l['user_id'] + '-' + l['review_id'] + "," + str(int(cat[0])) + "\n")

predictions.close()

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed: 27.6min finished


0.7314


In [21]:
model = linear_model.LogisticRegression(solver='sag', C=10, multi_class='auto', max_iter=1000, verbose=1)
model.fit(X_train, y_train)
pred = model.predict(X_validation)
prediction = pred == y_validation
print(sum(prediction) / len(prediction))

dataset_test = list(parseData("/home/cui/Projects/PycharmProjects/CSE-158/data/test_Category.json"))

predictions = open("/home/cui/Projects/PycharmProjects/CSE-158/data/predictions_Category_4000_sag.txt", 'w')
predictions.write("userID-reviewID,prediction\n")

for l in dataset_test:
    cat = model.predict([feature(l)])
    predictions.write(l['user_id'] + '-' + l['review_id'] + "," + str(int(cat[0])) + "\n")

predictions.close()

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


convergence after 78 epochs took 1064 seconds


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed: 17.7min finished


0.7348


In [17]:
model = linear_model.LogisticRegression(solver='sag', C=10, multi_class='auto', max_iter=2000, verbose=1)
model.fit(X_train, y_train)
pred = model.predict(X_validation)
prediction = pred == y_validation
print(sum(prediction) / len(prediction))

dataset_test = list(parseData("/home/cui/Projects/PycharmProjects/CSE-158/data/test_Category.json"))

predictions = open("/home/cui/Projects/PycharmProjects/CSE-158/data/predictions_Category_4300_sag.txt", 'w')
predictions.write("userID-reviewID,prediction\n")

for l in dataset_test:
    cat = model.predict([feature(l)])
    predictions.write(l['user_id'] + '-' + l['review_id'] + "," + str(int(cat[0])) + "\n")

predictions.close()

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


convergence after 76 epochs took 1121 seconds


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed: 18.7min finished


0.7368
